In [1]:
from libra.prompts import LLAMAINDEX_REACT_PROMPT, SYSTEM_INIT_PROMPT

In [2]:
from libra.models import ModelBackend, ModelFactory
from libra.types import ModelLabel
from libra.config import ChatGPTConfig
from libra.functions.retrieval_functions import RETRIEVAL_FUNCS, mb_information_retrieval, job_retrieval, mb_network_retrieval

loading embedding model...

/home/gcp_dpf/libra-core/.venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


finish!


In [3]:
model = ModelFactory.create(ModelLabel.GPT_4o, ChatGPTConfig(temperature=0.0).__dict__)

In [6]:
tools_desc = []

for func in RETRIEVAL_FUNCS:
    tool_desc = f"""
> Tool name: {func.get_function_name()}
Tool Description: {func.get_function_description()}
Tool Args: {func.get_openai_tool_schema()['function']['parameters']}
"""
    tools_desc.append(tool_desc)
    
    print(tool_desc)
    
tool_names = [func.get_function_name() for func in RETRIEVAL_FUNCS]

print(tool_names)


> Tool name: mb_information_retrieval
Tool Description: Retrieves MB Bank information based on a Vietnamese query.
Use when user ask anything about MB (MB Bank) such as: MB Bank's history,
MB Bank's products and services, MB Bank's achievement, MB Bank's officer salary, any problem relating to MB.  
Furthermore, if user want to receive advice about career path, use this function to add more context to your answer.
Tool Args: {'properties': {'query': {'type': 'string', 'description': 'A Vietnamese language query about MB Bank. You should generate this query as good as you can based on the conversation with user.'}}, 'required': ['query'], 'type': 'object'}


> Tool name: job_retrieval
Tool Description: Retrieve job opportunities at MB Bank based on user preferences.
This function should be used when:
- The user asks for information about job opportunities at MB Bank.
- The user describes their skills and wants to know if there are any suitable jobs at MB Bank.
- If the user ask about j

In [8]:

prompt = LLAMAINDEX_REACT_PROMPT.format(
    tool_desc = '\n'.join(tools_desc),
    tool_names = ', '.join(tool_names),
    context_prompt = SYSTEM_INIT_PROMPT
)

In [8]:
print(prompt)


Your are designed to specifically help with some tasks, from answering questions to providing summaries to other types of analyses, or
even executing some function to query a database or call an API.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:

> Tool name: mb_information_retrieval
Tool Description: Retrieves MB Bank information based on a Vietnamese query.
Use when user ask anything about MB (MB Bank) such as: MB Bank's history,
MB Bank's products and services, MB Bank's achievement, MB Bank's officer salary, any problem relating to MB.  
Furthermore, if user want to receive advice about career path, use this function to add more context to your answer.
Tool Args: {'properties': {'query': {'type': 'string', 'description': 'A Vie

In [49]:
input_text = "có job tư vấn nào ở Quảng Trị không "

response = model.run(messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": input_text}
])

text_response = ""
for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
        text_response += chunk.choices[0].delta.content

Thought: The current language of the user is Vietnamese. I need to use a tool to help me answer the question.
Action: job_retrieval
Action Input: {"workplace": "Quảng Trị", "industry": "Dịch vụ khách hàng"}

In [43]:
import json

lines = text_response.split("\n")

for line in lines:
    if line.startswith("Action: "):
        action = line.replace("Action: ", "")
    if line.startswith("Action Input: "):
        action_input = line.replace("Action Input: ", "")
        try:
            action_input = json.loads(action_input)
        except json.JSONDecodeError:
            raise ValueError("Invalid JSON in action input")
            

In [44]:
action, action_input

('mb_information_retrieval', {'query': 'MB Bank có hỗ trợ chơi hụi không?'})

In [45]:
results = globals()[action](**action_input) # type: ignore

print(results)


topic: None,
summary: Nhóm nghề nghiệp kinh doanh tại MB tiếp xúc với khách hàng và cung cấp các dịch vụ tài chính để đáp ứng nhu cầu của họ.
content: Nhóm nghề nghiệp kinh doanh
- Tính chất công việc: Tiếp xúc với khách hàng và cung cấp các dịch vụ tài chính để đáp ứng nhu cầu khách hàng của MB
            

topic: None,
summary: MB cung cấp dịch vụ ngân hàng thương mại cho khách hàng cá nhân và doanh nghiệp, bao gồm tiền gửi, ngân hàng số, cho vay, dịch vụ thẻ, chuyển tiền, bảo hiểm nhân thọ, và các dịch vụ tài chính khác.
content: Mô hình hoạt động của MB
MB thực hiện các loại hình của ngân hàng thương mại theo quy định của pháp luật và của Ngân hàng Nhà nước Việt Nam theo Giấy phép thành lập và hoạt động Ngân hàng thương mại cổ phần số 100/GP-NHNN ngày 16/10/2018, bao gồm các hoạt động chính phục vụ: 
- KHÁCH HÀNG CÁ NHÂN Mảng dịch vụ Khách hàng Cá nhân mang tới các dịch vụ toàn diện hướng tới phục vụ và thiết kế cho từng khách hàng. Mục tiêu của MB là mang đến trải nghiệm dịch vụ

In [46]:
messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": input_text},
    {"role": "assistant", "content": text_response},
    {"role": "user", "content": f"Observation:  tài liệu được tìm thấy: {results}"}
]

response = model.run(messages) # type: ignore

text_response = ""
for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end='')
        text_response += chunk.choices[0].delta.content

Thought: Theo thông tin từ tài liệu, MB Bank cung cấp nhiều dịch vụ tài chính đa dạng nhưng không có thông tin cụ thể về việc hỗ trợ chơi hụi. 

Answer: Hiện tại, MB Bank không có thông tin cụ thể về việc hỗ trợ chơi hụi. MB Bank chủ yếu cung cấp các dịch vụ tài chính như tiền gửi, ngân hàng số, cho vay, dịch vụ thẻ, chuyển tiền, bảo hiểm nhân thọ và các dịch vụ tài chính khác. Nếu bạn cần thêm thông tin chi tiết, mình khuyên bạn nên liên hệ trực tiếp với MB Bank qua các kênh hỗ trợ khách hàng như tổng đài MB247, email, hoặc chi nhánh/phòng giao dịch gần nhất nhé!